In [1]:

import pandas as pd
import numpy as np
import random
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [2]:

hs300=pd.read_csv(r'C:\Users\yanyifei\Desktop\HS300\biye\hs3001.csv',index_col ='A股股票代码_A_Stkcd')

In [3]:
X=hs300.iloc[:,2:-1]

In [4]:
X=(X.max()-X)/(X.max()-X.min())

In [5]:
pca = PCA(n_components=20)
X_p= pca.fit(X).transform(X)

print(X.shape)#原始变量维度

print(X_p.shape)#主成分变量的维度

(2332, 145)
(2332, 20)


In [6]:
X_PP=pd.DataFrame(X_p)

In [8]:
dd=X_PP

In [8]:
def cut(x):
    return pd.cut(x,5,labels = range(5))

In [9]:
dd=X_PP.apply(cut,axis=0)

In [10]:
dd=dd.astype(float)

In [10]:
y=hs300.iloc[:,-1]

In [11]:
y.index=range(len(y))
y

0       1
1       1
2       0
3       0
4       0
       ..
2327    0
2328    0
2329    1
2330    1
2331    0
Name: target, Length: 2332, dtype: int64

In [12]:
X_train, X_test, y_train, y_test = dd.iloc[:1642,],dd.iloc[1642:,],y.iloc[:1642,],y.iloc[1642:,]

In [19]:
x_train, x_test, y_train, y_test = train_test_split(dd, y.values)

In [39]:
class GeneticNeuralNetwork(Sequential):
 # 构造函数
    def __init__(self, child_weights=None):
 # 初始化顺序模型超类
        super().__init__()
 # 如果没有提供权重，则随机生成它们
        if child_weights is None:
 # 创建并随机生成层
            layer1 = Dense(16, input_shape=(20,), activation='relu')
            layer2 = Dense(32, activation='relu')

            layer3 = Dense(1, activation='sigmoid')
 # 图层将添加到模型中
            self.add(layer1)
            self.add(layer2)
            
            self.add(layer3)
 # 如果提供了权重，则在层内设置权重
        else:
 # 在层中设置权重
            self.add(
            Dense(
                16,
                input_shape=(20,),
                activation='relu',
                weights=[child_weights[0], np.zeros(16)])
                )
            self.add(
            Dense(
            32,
            activation='relu',
            weights=[child_weights[1], np.zeros(32)])
            )
            
            self.add(
            Dense(
            1,
            activation='sigmoid',
            weights=[child_weights[2], np.zeros(1)])
            )
 # 矩阵行向量的前向传播函数
    def forward_propagation(self, X_train, y_train,X_test,y_test):
    # 前向传播
        y_hat = self.predict(X_train.values)
    # 计算适应度得分
        self.fitness = accuracy_score(y_train, y_hat.round())
# 标准反向传播
    def compile_train(self, epochs):
        self.compile(optimizer=tf.keras.optimizers.SGD(nesterov=True),
        loss='binary_crossentropy',
        metrics=['accuracy']
        )
        self.fit(X_train.values, y_train.values,epochs=epochs,validation_data=( X_test.values, y_test.values))
 # 两个遗传神经网络的交叉特性


In [40]:
def dynamic_crossover(nn1, nn2):
# 各重量清单
    nn1_weights = []
    nn2_weights = []
    child_weights = []
    # 从第一个网络的所有层获取所有权重
    for layer in nn1.layers:
        nn1_weights.append(layer.get_weights()[0])
    # 从第二个网络的所有层获取所有权重
    for layer in nn2.layers:
        nn2_weights.append(layer.get_weights()[0])
    # 迭代所有层的所有权重进行交叉
    for i in range(0, len(nn1_weights)):
    # 基于列的单点分割父矩阵
        split = random.randint(0, np.shape(nn1_weights[i])[1]-1)
    # 在单个点之后迭代，并将reming cols设置为nn_2
        for j in range(split, np.shape(nn1_weights[i])[1]-1):
            nn1_weights[i][:, j] = nn2_weights[i][:, j]
    # 交叉后给孩子增加重量
        child_weights.append(nn1_weights[i])
    # 增加突变的机会
    mutation(child_weights)
    # 创建并返回子对象ct
    child = GeneticNeuralNetwork(child_weights)
    return child
    # 改变重量的机会


In [41]:
def mutation(child_weights):
    # 增加随机突变的机会
    selection = random.randint(0, len(child_weights)-1)
    mut = random.uniform(0, 1)
    if mut >= .5:
        child_weights[selection] *= random.randint(2, 5)
    else:
    # No mutation
        pass

In [42]:
networks = []
pool = []
# 世代追踪
generation = 0
# 初始人口
n = 20
# 生成n个随机加权神经网络
for i in range(0, n):
    networks.append(GeneticNeuralNetwork())
# 缓存最大适应值
max_fitness = 0
# 最大适应度得分
optimal_weights = []
# 进化回路
while max_fitness < .66:
    # 记录当前生成
    generation = generation+1
    print('Generation: ', generation)
    # 前向传播神经网络计算适应度得分
    for nn in networks:
    # 计算传播计算适应度得分
        nn.forward_propagation(X_train, y_train,X_test,y_test)
    # 计算适应度增加到池
        pool.append(nn)
    # 为下一个孩子的繁殖而清除
    networks.clear()
    # 根据适合度排序
    pool = sorted(pool, key=lambda x: x.fitness)
    pool.reverse()
    # 找到最大适合度并记录相关权重
    for i in range(0, len(pool)):
    # 如果在人群中有一个新的最大适应度
        if pool[i].fitness > max_fitness:
            max_fitness = pool[i].fitness
            print('Max Fitness: ', max_fitness)
    # 重置最佳加权
            optimal_weights = []
    # 迭代所有层，获取权重，并附加到最优值
    for layer in pool[i].layers:
        optimal_weights.append(layer.get_weights()[0])
#     print('optimal_weights',optimal_weights)
    #交叉，前5名随机选择2个伙伴为孩子
    for i in range(0, 5):
        for j in range(0, 2):
    # 创建子项并添加到网络
            temp = dynamic_crossover(pool[i], random.choice(pool))
            # 添加到网络以计算下一次迭代的适应度得分
            networks.append(temp)
# 用遗传神经网络建立初始权值
gnn = GeneticNeuralNetwork(optimal_weights)
gnn.compile_train(20)
# 样本外遗传神经网络的检验

Generation:  1
Max Fitness:  0.6540803897685749
Generation:  2
Max Fitness:  0.6577344701583435
Generation:  3
Generation:  4
Generation:  5
Max Fitness:  0.6583434835566383
Generation:  6
Generation:  7
Generation:  8
Max Fitness:  0.658952496954933
Generation:  9
Max Fitness:  0.6595615103532277
Generation:  10
Generation:  11
Generation:  12
Generation:  13
Generation:  14
Generation:  15
Generation:  16
Max Fitness:  0.6632155907429963
Epoch 1/20
52/52 [==============================] - 0s 4ms/step - loss: 0.7490 - accuracy: 0.4013 - val_loss: 0.7009 - val_accuracy: 0.5116
Epoch 2/20
52/52 [==============================] - 0s 1ms/step - loss: 0.6889 - accuracy: 0.5560 - val_loss: 0.6695 - val_accuracy: 0.6145
Epoch 3/20
52/52 [==============================] - 0s 1ms/step - loss: 0.6650 - accuracy: 0.6248 - val_loss: 0.6569 - val_accuracy: 0.6580
Epoch 4/20
52/52 [==============================] - 0s 1ms/step - loss: 0.6545 - accuracy: 0.6437 - val_loss: 0.6501 - val_accuracy: 0.6

In [43]:
y_hat = gnn.predict(X_test.values)
print('Test Accuracy: %.2f' % accuracy_score(y_test, y_hat.round()))

Test Accuracy: 0.68


In [39]:
time1_str

'2020-11-08 21:07:36'